In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from eden.converter.molecule import obabel
import networkx as nx
import pybel
import requests
import os.path

In [3]:
def get_compounds(fname, size, listkey):
    PROLOG='https://pubchem.ncbi.nlm.nih.gov/rest/pug/'
    with open(fname,'w') as file_handle:
        stepsize=50
        index_start=0
        for chunk, index_end in enumerate(range(0,size+stepsize,stepsize)):
            if index_end is not 0 :
                print 'Chunk %s) Processing compounds %s to %s (of a total of %s)' % (chunk, index_start, index_end-1, size)
                RESTQ = PROLOG + 'compound/listkey/' + str(listkey) + '/SDF?&listkey_start=' + str(index_start) + '&listkey_count=' + str(stepsize)
                reply=requests.get(RESTQ)
                file_handle.write(reply.text)
            index_start = index_end
        print 'compounds available in file: ', fname


def get_assay(assay_id):
    PROLOG='https://pubchem.ncbi.nlm.nih.gov/rest/pug/'
    AID=str(assay_id)
    #active
    RESTQ = PROLOG + 'assay/aid/' + AID + '/cids/JSON?cids_type=active&list_return=listkey'
    reply=requests.get(RESTQ)
    #extract the listkey
    active_listkey = reply.json()['IdentifierList']['ListKey']
    active_size = reply.json()['IdentifierList']['Size'] 
    active_fname = 'data/AID'+AID+'_active.sdf'
    get_compounds(fname=active_fname, size=active_size, listkey=active_listkey)

    #inactive
    RESTQ = PROLOG + 'assay/aid/' + AID + '/cids/JSON?cids_type=inactive&list_return=listkey'
    reply=requests.get(RESTQ)
    #extract the listkey
    inactive_listkey = reply.json()['IdentifierList']['ListKey']
    inactive_size = reply.json()['IdentifierList']['Size']
    inactive_fname = 'data/AID'+AID+'_inactive.sdf'
    get_compounds(fname=inactive_fname, size=inactive_size, listkey=inactive_listkey)

    return (active_fname,inactive_fname)

In [4]:
%%time

AID=825
READ_FROM_FILE=True
DATA_DIR = 'data'
if READ_FROM_FILE:
    active_fname=DATA_DIR + '/AID%s_active.sdf'%AID
    inactive_fname=DATA_DIR + '/AID%s_inactive.sdf'%AID
else:
    active_fname, inactive_fname = get_assay(AID)


CPU times: user 10 µs, sys: 0 ns, total: 10 µs
Wall time: 14.1 µs


Generate conformers for data

In [5]:
%%time
active_conf = DATA_DIR + '/conf_AID%s_active.sdf'%AID
inactive_conf = DATA_DIR + '/conf_AID%s_inactive.sdf'%AID
# Active compounds
obabel.generate_conformers(active_fname, active_conf, 10, 'rmsd')
# Inactve compounds
obabel.generate_conformers(active_fname, inactive_conf, 10, 'rmsd')

['obabel', 'data/AID825_active.sdf', '-O', 'data/conf_AID825_active.sdf', '--conformer', '--nconf', '10', '--score', 'rmsd', '--writeconformers']
['obabel', 'data/AID825_active.sdf', '-O', 'data/conf_AID825_inactive.sdf', '--conformer', '--nconf', '10', '--score', 'rmsd', '--writeconformers']
CPU times: user 1.98 ms, sys: 9.4 ms, total: 11.4 ms
Wall time: 20.8 ms


Functions for training and testing the model

In [9]:
import datetime, time
def train_obabel_model(pos_fname, neg_fname, model_type = "default",
                       model_fname=None, n_iter=40, active_set_size=1000, 
                       n_active_learning_iterations=3, threshold=1, train_test_split=0.7, 
                       verbose=False):
    # this will be passed as an argument to the model later on
    def pre_processor( data, **args):
        return data
    
    from eden.graph import Vectorizer
    vectorizer = Vectorizer()

    from sklearn.linear_model import SGDClassifier
    estimator = SGDClassifier(class_weight='auto', shuffle=True)

    #create iterable from files
    from eden.converter.molecule import obabel
    if model_type == "default":
        iterable_pos=obabel.obabel_to_eden(pos_fname)
        iterable_neg=obabel.obabel_to_eden(neg_fname)
    elif model_type == "3d":
        iterable_pos=obabel.obabel_to_eden3d(pos_fname)
        iterable_neg=obabel.obabel_to_eden3d(neg_fname)

    
    from itertools import tee
    iterable_pos, iterable_pos_ = tee(iterable_pos)
    iterable_neg, iterable_neg_ = tee(iterable_neg)
    
    import time
    start = time.time()
    print('# positives: %d  # negatives: %d (%.1f sec %s)'%(sum(1 for x in iterable_pos_), sum(1 for x in iterable_neg_), time.time() - start, str(datetime.timedelta(seconds=(time.time() - start)))))
    
    #split train/test
    from eden.util import random_bipartition_iter
    iterable_pos_train, iterable_pos_test = random_bipartition_iter(iterable_pos, relative_size=train_test_split)
    iterable_neg_train, iterable_neg_test = random_bipartition_iter(iterable_neg, relative_size=train_test_split)



    #make predictive model
    from eden.model import ActiveLearningBinaryClassificationModel
    model = ActiveLearningBinaryClassificationModel( pre_processor, estimator=estimator, vectorizer=vectorizer )

    #optimize hyperparameters and fit model
    from numpy.random import randint
    from numpy.random import uniform

    pre_processor_parameters={} 

    vectorizer_parameters={'complexity':[4]}

    estimator_parameters={'n_iter':randint(5, 100, size=n_iter),
                          'penalty':['l1','l2','elasticnet'],
                          'l1_ratio':uniform(0.1,0.9, size=n_iter), 
                          'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                          'power_t':uniform(0.1, size=n_iter),
                          'alpha': [10**x for x in range(-8,-2)],
                          'eta0': [10**x for x in range(-4,-1)],
                          'learning_rate': ["invscaling", "constant", "optimal"]}

    model.optimize(iterable_pos_train, iterable_neg_train, 
                   model_name=model_fname,
                   n_active_learning_iterations=n_active_learning_iterations,
                   size_positive=-1,
                   size_negative=active_set_size,
                   n_iter=n_iter, cv=3, n_jobs=1, verbose=verbose,
                   pre_processor_parameters=pre_processor_parameters, 
                   vectorizer_parameters=vectorizer_parameters, 
                   estimator_parameters=estimator_parameters)
  
    #estimate predictive performance
    model.estimate( iterable_pos_test, iterable_neg_test, cv=5 )
    return model

def test_obabel_model(fname, model_type = "default", model_fname=None):
    from eden.model import ActiveLearningBinaryClassificationModel

    model = ActiveLearningBinaryClassificationModel()
    model.load(model_fname)

    #create iterable from files
    from eden.converter.molecule import obabel
    if model_type == "default":
        iterable=obabel.obabel_to_eden(fname)
    elif model_type == "3d":
        iterable=obabel.obabel_to_eden3d(fname)
    
    predictions= model.decision_function( iterable )
        
    return predictions

Train the model

In [10]:

%%time
model_fname='AID%s.model'%AID
model = train_obabel_model(active_fname, inactive_fname,
                           model_type = "3d",
                           model_fname=model_fname, 
                           n_iter=40, 
                           active_set_size=500, 
                           n_active_learning_iterations=4, 
                           threshold=1, 
                           train_test_split=0.8, 
                           verbose=1)

# positives: 48  # negatives: 50 (9.0 sec 0:00:09.019055)
Parameters range:
Pre_processor:
{}
Vectorizer:
{'complexity': [4]}
Estimator:
{'alpha': [1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001],
 'eta0': [0.0001, 0.001, 0.01],
 'l1_ratio': array([ 0.38381386,  0.5982971 ,  0.1625781 ,  0.72375233,  0.8909441 ,
        0.43001992,  0.65570149,  0.35000627,  0.43198795,  0.55473367,
        0.16338035,  0.75319608,  0.87788697,  0.38330997,  0.79427822,
        0.8555691 ,  0.27357025,  0.63396179,  0.62453756,  0.22191049,
        0.34484502,  0.37903195,  0.22159867,  0.21118245,  0.69108203,
        0.88935332,  0.71709542,  0.15740611,  0.85390072,  0.71429862,
        0.45871043,  0.67315424,  0.25851083,  0.66201299,  0.59022684,
        0.75313672,  0.36740917,  0.62811959,  0.47007623,  0.65685486]),
 'learning_rate': ['invscaling', 'constant', 'optimal'],
 'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
 'n_iter': array([25, 40, 86, 66, 10, 48, 86, 69, 28

/Users/jl/.local/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/jl/.local/lib/python2.7/site-packages/numpy/core/_methods.py:71: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


KeyError: 'vector'